In [ ]:
#!pip install pydrive

In [1]:
import os

In [2]:
contenido = os.listdir(r'D:\PYTHON\utea_scripts\reportes_credito_siembra\pdfs')

dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
dic_files

{12478: '12478_SERRATE COLAMARINO HUGO.pdf',
 18027: '18027_AGUILERA TARRADELLES GUILLERMO.pdf',
 2029: '2029_COLAMARINO DI SILVIO ANGELO.pdf',
 2392: '2392_COLAMARINO SUAREZ DOMENICO A..pdf',
 2408: '2408_CHOQUE CACERES FREDDY.pdf',
 2662: '2662_DON SERAFIN SRL..pdf',
 40139: '40139_ROCA SUAREZ MARIA ALEJANDRA.pdf',
 41820: '41820_ASUBY S.R.L..pdf',
 633: '633_AGROP. VICENTE ROCA GIL SRL.pdf',
 743: '743_ARTEAGA CHAVEZ ROBERT E..pdf'}

#  CARGAR FILES A DRIVE

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [4]:
print(len(dic_files), 'INFORMES')
dic_files

10 INFORMES


{12478: '12478_SERRATE COLAMARINO HUGO.pdf',
 18027: '18027_AGUILERA TARRADELLES GUILLERMO.pdf',
 2029: '2029_COLAMARINO DI SILVIO ANGELO.pdf',
 2392: '2392_COLAMARINO SUAREZ DOMENICO A..pdf',
 2408: '2408_CHOQUE CACERES FREDDY.pdf',
 2662: '2662_DON SERAFIN SRL..pdf',
 40139: '40139_ROCA SUAREZ MARIA ALEJANDRA.pdf',
 41820: '41820_ASUBY S.R.L..pdf',
 633: '633_AGROP. VICENTE ROCA GIL SRL.pdf',
 743: '743_ARTEAGA CHAVEZ ROBERT E..pdf'}

In [5]:
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "1RA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(r'D:\PYTHON\utea_scripts\reportes_credito_siembra\pdfs//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 12478_SERRATE COLAMARINO HUGO.pdf with mimeType application/pdf
Created file 18027_AGUILERA TARRADELLES GUILLERMO.pdf with mimeType application/pdf
Created file 2029_COLAMARINO DI SILVIO ANGELO.pdf with mimeType application/pdf
Created file 2392_COLAMARINO SUAREZ DOMENICO A..pdf with mimeType application/pdf
Created file 2408_CHOQUE CACERES FREDDY.pdf with mimeType application/pdf
Created file 2662_DON SERAFIN SRL..pdf with mimeType application/pdf
Created file 40139_ROCA SUAREZ MARIA ALEJANDRA.pdf with mimeType application/pdf
Created file 41820_ASUBY S.R.L..pdf with mimeType application/pdf
Created file 633_AGROP. VICENTE ROCA GIL SRL.pdf with mimeType application/pdf
Created file 743_ARTEAGA CHAVEZ ROBERT E..pdf with mimeType application/pdf
end


In [6]:
print(len(dic_urls), 'URLs')
dic_urls

10 URLs


{12478: 'https://drive.google.com/file/d/1rk26B9a5M8IvQ6pC_U5nKpqcwtLlCTJu/view?usp=drivesdk',
 18027: 'https://drive.google.com/file/d/1wnCAUBDZzVLvl3dOTkcs1sThX0b_4aZC/view?usp=drivesdk',
 2029: 'https://drive.google.com/file/d/1c2PelYsk7JtpRc0M9rX7ZlV8N8ZA1jZB/view?usp=drivesdk',
 2392: 'https://drive.google.com/file/d/13Z4VaOapQ5PQgtW0ipWQbIh4LsMuCYp_/view?usp=drivesdk',
 2408: 'https://drive.google.com/file/d/1-7n_aEdrmY-lbKGIVj3cl6C8JibgLRE-/view?usp=drivesdk',
 2662: 'https://drive.google.com/file/d/1VUTd0sR2HYJVT3WoG4wfnsnWilUCBbKd/view?usp=drivesdk',
 40139: 'https://drive.google.com/file/d/1waBi9mqk551HK3NR4RIc8sU6bHvsKYbT/view?usp=drivesdk',
 41820: 'https://drive.google.com/file/d/1PRrOnnXAR8ghZ3GleZ41BkiU2P_JTksv/view?usp=drivesdk',
 633: 'https://drive.google.com/file/d/1a-5bFEqYLcYwxRQXBBoW6Ids-ipMzT1d/view?usp=drivesdk',
 743: 'https://drive.google.com/file/d/19hC2JaPq5N31XJdUGyPtuQcenepjRkZW/view?usp=drivesdk'}

# CONECTAR A AMIGOCLOUD

In [8]:
from amigocloud import AmigoCloud

In [11]:
# CONECTAR A AMIGOCLOUD
acceso = open('../../../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [12]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_1ra = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')

12478
18027
2029
2392
2408
2662
40139
41820
633
743
end


# -----------------------------------------------------------

# CONECCION A DRIVE

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "REPORTES_CREDITO"):
      fileID = file['id']

file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
file1.SetContentFile("file.pdf")
file1.Upload() # Upload the file.
print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))

In [ ]:
permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})

print(file1['alternateLink'])  # Display the sharable link.

In [ ]:
for file in fileList[0]:
    print(file, '       ',fileList[0][file])

In [ ]:
fileList[0]['title']